<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Cristina Vazquez, Gerardo Ruiz Velasco Lazalde.

**Fecha:** 7 de mayo del 2021.

**Expediente** : 721215 ,715878 .
**Profesor:** Oscar David Jaramillo Zuluaga.

**Link repositorio: https://github.com/GerardoRVL97/TAREA_11_RUIZ_G_VAZQUEZ_C **

# Tarea 11: Clase:24 Valuacion Opciones Barrera

# Tarea
Realizar la valuación de las opciones barrera para las acciones de Microsoft, siguiendo los siguientes lineamientos:
1. Realizar la valuación para el tipo de opción barrera 'CALL'- down-and-out, usando la suposición de rendimientos normales y usando el método del histograma. Para ambas valuaciones, comparar los resultados de la evaluación obtenida con los resultados obtenidos por la valuación Europea. Como se realizó en clase.
2. Repetir el inciso anterior pero en esta ocasión para el caso de 'PUT'- Up-and-out.

**Nota:** Analice los datos descargados y según el método ya sea `down-and-out` o `Up-and-out` defina un strike price usando la volatility smile vista en la clase de opciones europeas y basado en el spot price y el strike defina una barrera adecuada.

Se habilitará un enlace en canvas para subir un cuaderno de python, con la solución.

**Criterio de revisión.**

> Se te calificará de acuerdo a los resultados finales que reportes, basados en tu análisis.

In [ ]:
def Riemann_approach_Trap(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',tipo:'call o put',
                    flag=None):
    t_emp = time.time()
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Creamos el segundo DataFrame
    mult= pd.DataFrame(2+r*(T/NbStep)+np.random.randn(NbTraj,NbStep)*sigma).T
    # Multiplicamos los dataframes
    prices_mult= prices*mult
    # Hacemos la sumatoria
    sumprices_mult = np.cumsum(prices_mult)
    # Multiplicamos por h/2 
    sumatoria = sumprices_mult*(1/NbStep)*(1/2)*(1/T)
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    if tipo =='call':
        # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
        valor = pd.DataFrame({'Prima':np.exp(-r*T) \
                             *np.fmax(sumatoria-strike,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
        # intervalos de confianza
        #confianza = 0.95
        #sigma_est = call.sem().Prima
        #mean_est = call.iloc[-1].Prima
        #i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
        #return call.iloc[-1].Prima, i1
    else:
        valor = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(strike-sumatoria,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
        # intervalos de confianza
    confianza = 0.95
    sigma_est = valor.sem().Prima
    mean_est = valor.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    dif95 = np.round(i1[1] - i1[0],4)
    t_acab = time.time()
    e = int(t_acab - t_emp)
    time_format = '{:02d}:{:02d}'.format((e % 3600 // 60), e % 60)
    tiempo = lambda x: '< 1 s' if x == '00:00' else x
    return np.array([np.round(valor.iloc[-1].Prima,4),np.round(i1[0],4),np.round(i1[1],4),dif95,
                     tiempo(time_format)])
    #return .iloc[-1].Prima, i1